In [ ]:
# random generation of genomes; distribution of dcj operations needed 

# one or two duplicated genes; just sample different deduplication permutations -- measure variability in dcj distance 
    # min and max 
    # probability distribution 
    # want to show there's a huge variability betewen what map you choose 

# rm and ls 

# nature one 